In [1]:
import os
from dotenv import load_dotenv
from mail_helper import *
from helper_functions import *

load_dotenv()
YAHOO_MAIL_APP_PASSWORD = os.getenv("YAHOO_MAIL_APP_PASSWORD")
YAHOO_IMAP_USER = os.getenv("YAHOO_IMAP_USER")
YAHOO_IMAP_SERVER = os.getenv("YAHOO_IMAP_SERVER")
YAHOO_IMAP_PORT = os.getenv("YAHOO_IMAP_PORT")

In [20]:
email_data = fetch_emails(YAHOO_IMAP_SERVER, YAHOO_IMAP_USER, YAHOO_MAIL_APP_PASSWORD, limit=50)
email_data['47']

{'ID': '47',
 'From': 'admissions@sicsr.ac.in',
 'Subject': 'Last 5 days left to register for SNAP and be  part of MBA programs at\r\n SICSR, Pune',
 'Date': '2021-11-22T14:07:20+00:00'}

In [3]:
# email_ids_to_delete = ['50']
# delete_specific_emails(YAHOO_IMAP_SERVER, YAHOO_IMAP_USER, YAHOO_MAIL_APP_PASSWORD, email_ids_to_delete)

In [4]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(
        model="gemini-1.0-pro",
        google_api_key=GEMINI_API_KEY)

In [5]:
from langchain.tools import BaseTool

class DeleteMail(BaseTool):
    name = "delete_mail"
    description = "Use this tool to delete mail with a given id. It requires a mail id as input to delete mail. Use it to delete mails. Be sure syntax is correct."

    def _run(self, email_id):
      try:
        return delete_specific_email(YAHOO_IMAP_SERVER, YAHOO_IMAP_USER, YAHOO_MAIL_APP_PASSWORD, email_id)
      except Exception:
        return "This is not a vaild input. Try a different syntax."

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

delete_tool = DeleteMail()
tools = [delete_tool]

In [16]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts.prompt import PromptTemplate

template_prompt = '''
You are given the subject of a mail and it's email id in the following format:
subject of the mail is :"subject of the mail"  and email_id is :"a unique email_id associated with a given mail"

Based on this subject classify whether the given mail is a promotional mail or not given its subject. If it is a promotional mail, then delete it using the following tool, otherwise leave the mail as it is:
{tools}

Use the following format:

Question: is the input subject, a subject of the promotional mail?
Thought: you should think if this subject promotes a product or a service.
Action: the action to take, should be either to delete using the tool [{tool_names}] or leave it as it is.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer whether the mails was deleted or not

Begin!

Question:{input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template=template_prompt
)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [17]:
input_variable = f"subject of the mail is :{email_data['47']['Subject']} and email_id is :{email_data['47']['ID']}"
input_variable

'subject of the mail is :Only 5 days left! A promising management career awaits! Register now\r\n for SCMHRD and email_id is :47'

In [19]:
email_data['47']

{'ID': '47',
 'From': 'admissions@scmhrd.edu',
 'Subject': 'Only 5 days left! A promising management career awaits! Register now\r\n for SCMHRD',
 'Date': '2021-11-22T13:09:16+00:00'}

In [18]:
agent_executor.invoke({"input": input_variable})



> Entering new AgentExecutor chain...
Yes, it promtes SCMHRD.
Action: delete_mail
Action Input: 47Email with ID 47 deletedI now know the final answer
Final Answer: Deleted

> Finished chain.


{'input': 'subject of the mail is :Only 5 days left! A promising management career awaits! Register now\r\n for SCMHRD and email_id is :47',
 'output': 'Deleted'}

In [6]:
# from langchain import hub
# prompt = hub.pull("hwchase17/react")
# print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
